# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [13]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter 
import math 
from scipy import stats 

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [14]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [15]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 88172896,
 'calculationPrice': 'close',
 'change': 3.46,
 'changePercent': 0.02496,
 'close': 145.18,
 'closeSource': 'fioifcla',
 'closeTime': 1711510679383,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 142.24,
 'delayedPriceTime': 1716662133234,
 'extendedChange': 0.08,
 'extendedChangePercent': 0.00057,
 'extendedPrice': 147.47,
 'extendedPriceTime': 1663126601233,
 'high': 145.72,
 'highSource': 'iape1iryd5 utn lde emce',
 'highTime': 1665552368585,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 148.852,
 'iexCloseTime': 1711191263410,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 146.06,
 'iexOpenTime': 1719948017004,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1686517692629,
 'latestPrice': 145.79,
 'latestSource': 'Close',
 'latestTime': 'June 24, 2022',
 'latestUpdate': 1707172203952,
 'latestVolume': 89582934,
 'low': 144.

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [16]:
pe_ratio = data['peRatio']
pe_ratio

23.28

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [17]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [18]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_6133/4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_6133/4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_6133/4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_6133/4032552586.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,124,29.53,N/A
1,AAL,14.4,-3.89,N/A
2,AAP,186.4,20.44,N/A
3,AAPL,145.6,23.12,N/A
4,ABBV,159.33,22.25,N/A
...,...,...,...,...
496,YUM,116.31,21.53,N/A
497,ZBH,108.92,110.77,N/A
498,ZBRA,312.1,21.09,N/A
499,ZION,54.84,5.64,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [23]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.91,2.45,N/A
1,EBAY,46.23,2.6,N/A
2,AMZN,116.57,2.9,N/A
3,NUE,115.41,4.09,N/A
4,SYF,30.94,4.18,N/A
5,F,12.27,4.29,N/A
6,COF,112.96,4.47,N/A
7,PVH,66.36,4.66,N/A
8,AIG,53.27,4.69,N/A
9,DHI,69.43,4.93,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [19]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [25]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [26]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,38.91,2.45,514
1,EBAY,46.23,2.6,432
2,AMZN,116.57,2.9,171
3,NUE,115.41,4.09,173
4,SYF,30.94,4.18,646
5,F,12.27,4.29,1629
6,COF,112.96,4.47,177
7,PVH,66.36,4.66,301
8,AIG,53.27,4.69,375
9,DHI,69.43,4.93,288


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [ ]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [27]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_6133/682189717.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_6133/682189717.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_6133/682189717.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append(
/var/folders/y4/yphzsd_j3hbd5rzqwpdq7x9m0000gn/T/ipykernel_6133/682189717.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.

In [28]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.91,N/A,29.69,N/A,7.08,N/A,5.62,N/A,21.066512,N/A,10.93449,N/A,N/A
1,AAL,14.1,N/A,-3.92,N/A,-1.06,N/A,0.2606,N/A,-22.324626,N/A,1.002723,N/A,N/A
2,AAP,187.2,N/A,20.56,N/A,3.85,N/A,1.06,N/A,11.653391,N/A,2.510234,N/A,N/A
3,AAPL,142.95,N/A,23.68,N/A,34.2,N/A,5.96,N/A,17.638573,N/A,13.909126,N/A,N/A
4,ABBV,154.21,N/A,22.46,N/A,17.31,N/A,4.97,N/A,11.515416,N/A,8.351763,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,119.39,N/A,21.31,N/A,-3.94,N/A,5.1,N/A,19.375025,N/A,8.546622,N/A,N/A
497,ZBH,113.8,N/A,113.13,N/A,2,N/A,3.11,N/A,18.208094,N/A,5.413791,N/A,N/A
498,ZBRA,321.91,N/A,20.51,N/A,5.82,N/A,2.96,N/A,15.871763,N/A,6.471837,N/A,N/A
499,ZION,55.15,N/A,5.64,N/A,1.38,N/A,2.33,N/A,4.502533,N/A,2.198249,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [29]:

rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,189.7,N/A,61.61,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,278.43,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,278.73,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,98.8,N/A,48.66,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11,N/A,10.08,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,25.29,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,50.71,N/A,14.48,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,59.99,N/A,32.85,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,30.96,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRT,100.4,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [30]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [31]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [32]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.714571
1      0.033932
2      0.493014
3      0.588822
4      0.540918
         ...   
496    0.516966
497    0.984032
498    0.491018
499    0.077844
500    0.869261
Name: PE Percentile, Length: 501, dtype: object
0      0.810379
1       0.05988
2      0.572854
3      0.968064
4      0.932136
         ...   
496    0.055888
497    0.307385
498    0.748503
499    0.167665
500    0.934132
Name: PB Percentile, Length: 501, dtype: object
0      0.818363
1      0.011976
2      0.161677
3      0.832335
4      0.773453
         ...   
496    0.793413
497     0.55489
498    0.532934
499    0.421158
500     0.94012
Name: PS Percentile, Length: 501, dtype: object
0      0.806387
1      0.007984
2      0.387226
3      0.698603
4      0.379242
         ...   
496    0.760479
497    0.722555
498    0.640719
499     0.07984
500    0.922156
Name: EV/EBITDA Percentile, Length: 501, dtype: object
0      0.804391
1       0.03992
2      0.149701
3      0.886228
4      0.666667
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.91,N/A,29.69,0.714571,7.08,0.810379,5.6200,0.818363,21.066512,0.806387,10.934490,0.804391,N/A
1,AAL,14.1,N/A,-3.92,0.033932,-1.06,0.05988,0.2606,0.011976,-22.324626,0.007984,1.002723,0.03992,N/A
2,AAP,187.2,N/A,20.56,0.493014,3.85,0.572854,1.0600,0.161677,11.653391,0.387226,2.510234,0.149701,N/A
3,AAPL,142.95,N/A,23.68,0.588822,34.20,0.968064,5.9600,0.832335,17.638573,0.698603,13.909126,0.886228,N/A
4,ABBV,154.21,N/A,22.46,0.540918,17.31,0.932136,4.9700,0.773453,11.515416,0.379242,8.351763,0.666667,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,119.39,N/A,21.31,0.516966,-3.94,0.055888,5.1000,0.793413,19.375025,0.760479,8.546622,0.678643,N/A
497,ZBH,113.8,N/A,113.13,0.984032,2.00,0.307385,3.1100,0.55489,18.208094,0.722555,5.413791,0.403194,N/A
498,ZBRA,321.91,N/A,20.51,0.491018,5.82,0.748503,2.9600,0.532934,15.871763,0.640719,6.471837,0.489022,N/A
499,ZION,55.15,N/A,5.64,0.077844,1.38,0.167665,2.3300,0.421158,4.502533,0.07984,2.198249,0.135729,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [33]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,121.91,N/A,29.69,0.714571,7.08,0.810379,5.6200,0.818363,21.066512,0.806387,10.934490,0.804391,0.790818
1,AAL,14.1,N/A,-3.92,0.033932,-1.06,0.05988,0.2606,0.011976,-22.324626,0.007984,1.002723,0.03992,0.030739
2,AAP,187.2,N/A,20.56,0.493014,3.85,0.572854,1.0600,0.161677,11.653391,0.387226,2.510234,0.149701,0.352894
3,AAPL,142.95,N/A,23.68,0.588822,34.20,0.968064,5.9600,0.832335,17.638573,0.698603,13.909126,0.886228,0.79481
4,ABBV,154.21,N/A,22.46,0.540918,17.31,0.932136,4.9700,0.773453,11.515416,0.379242,8.351763,0.666667,0.658483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,119.39,N/A,21.31,0.516966,-3.94,0.055888,5.1000,0.793413,19.375025,0.760479,8.546622,0.678643,0.561078
497,ZBH,113.8,N/A,113.13,0.984032,2.00,0.307385,3.1100,0.55489,18.208094,0.722555,5.413791,0.403194,0.594411
498,ZBRA,321.91,N/A,20.51,0.491018,5.82,0.748503,2.9600,0.532934,15.871763,0.640719,6.471837,0.489022,0.580439
499,ZION,55.15,N/A,5.64,0.077844,1.38,0.167665,2.3300,0.421158,4.502533,0.07984,2.198249,0.135729,0.176447


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [34]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [37]:
portfolio_input()

In [38]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.1,14,-3.92,0.033932,-1.0600,0.05988,0.2606,0.011976,-22.324626,0.007984,1.002723,0.03992,0.030739
1,AIG,52.8,3,4.78,0.061876,0.7795,0.075848,0.8165,0.117764,2.875491,0.02994,0.798757,0.017964,0.060679
2,PVH,65.61,3,4.62,0.05988,0.8430,0.07984,0.4787,0.048902,4.532759,0.081836,1.037418,0.041916,0.062475
3,GM,34.84,5,5.83,0.083832,0.8566,0.08483,0.4083,0.033932,1.954385,0.01996,1.599838,0.08982,0.062475
4,GPS,9.76,20,-51.66,0.011976,1.4200,0.180639,0.2258,0.007984,5.037705,0.097804,0.756956,0.015968,0.062874
5,LNC,50.04,3,7.27,0.111776,0.5727,0.065868,0.4490,0.03992,5.078085,0.101796,0.440124,0.001996,0.064271
6,UNM,34.71,5,7.64,0.120758,0.6586,0.071856,0.6061,0.071856,3.956900,0.053892,0.570768,0.007984,0.065269
7,KSS,39.91,5,6.20,0.095808,1.1600,0.128743,0.2625,0.013972,3.851831,0.0499,1.114522,0.057884,0.069261
8,F,12.12,16,4.23,0.055888,1.1000,0.11976,0.3572,0.02994,3.687390,0.03992,1.863560,0.103792,0.06986
9,SYF,29.88,6,4.04,0.051896,1.1900,0.134731,0.9071,0.129741,2.448563,0.021956,0.903657,0.025948,0.072854


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()